In [ ]:
import pandas as pd
from PIL import Image, ImageEnhance, ImageDraw, ImageFont


# Media DataFrame to track files and edits
media_df = pd.DataFrame(columns=["filename", "type", "operations", "status"])

def add_media(file_path, media_type):
    """Register a new media file into the tracking DataFrame."""
    global media_df
    media_df.loc[len(media_df)] = [file_path, media_type, [], "raw"]

def apply_edit(index, operation, **params):
    """Apply an edit to a media file and update the DataFrame."""
    global media_df
    row = media_df.loc[index]
    
    if row["type"] == "image":
        img = Image.open(row["filename"])
        
        if operation == "brightness":
            enhancer = ImageEnhance.Brightness(img)
            img = enhancer.enhance(params.get("factor", 1.0))
        elif operation == "contrast":
            enhancer = ImageEnhance.Contrast(img)
            img = enhancer.enhance(params.get("factor", 1.0))
        elif operation == "sharpness":
            enhancer = ImageEnhance.Sharpness(img)
            img = enhancer.enhance(params.get("factor", 1.0))
        elif operation == "color":
            enhancer = ImageEnhance.Color(img)
            img = enhancer.enhance(params.get("factor", 1.0))
        elif operation == "text_overlay":
            draw = ImageDraw.Draw(img)
            font = ImageFont.load_default()
            draw.text((params.get("x", 10), params.get("y", 10)), 
                      params.get("text", "Sample"), 
                      fill=params.get("color", "black"), 
                      font=font)
        
        # Save edited version
        output_path = row["filename"].replace(".", "_edited.")
        img.save(output_path)
        
        # Update DataFrame with operation details
        media_df.at[index, "operations"] = row["operations"] + [(operation, params)]
        media_df.at[index, "status"] = "edited"
        media_df.at[index, "filename"] = output_path

def show_media_log():
    """Display the tracking DataFrame."""
    global media_df
    return media_df
